### This program corresponds to the fourth experiment of article, which solves the exterior problem of Helmholtz equation with different wave numbers

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import scipy.special as scp
from torch.autograd import Variable as v
import time
import math

is_gpu = torch.cuda.is_available()
if is_gpu:
    id = 0
    torch.cuda.set_device(id)
    
#gpu_nums = torch.cuda.device_count()
#gpu_index = torch.cuda.current_device()
#print(is_gpu,gpu_nums,gpu_index)
device = torch.device('cuda' if is_gpu else 'cpu')

device = torch.device('cpu')
print(device)
torch.set_default_tensor_type('torch.DoubleTensor')
print(torch.__version__)

import sys
sys.path.append('../Models')
sys.path.append('../Utils')
import Network as Nw
import BoundaryCondition as Bdc
import Loss_functions as lf
import DomainSetting as Ds
import GenerateIntegralMatrix as IM
import Training as Tn



cpu
1.6.0+cu101


In [153]:
# Set problem domain and boundary condition

# k                         The set of the wavenumbers
# k_num                     The number of the wavenumbers
# sample_num                The number of the sample points in boundary
# sample_x                  The coordinate of the sample points in boundary
# sample_u_r, sample_u_i    The boundary condition
# rprime                    The tangent vectors of sample points with parametric equation in boundary
# normal                     The exterior normal of the sample points in boundary
# sample_xk                 The input of the neural network
k_num = 60
k = torch.cat((torch.rand(20)*1.5+2,torch.rand(40)*1.5+4.5),0)
#torch.Tensor([2.0,4.0]) 

example = 'ex4-1'
if example == 'ex4-1':
    domain_type = 'smoothdomain'
    problem_type ='Helmholtz'
    in_or_out = 'out'
    layer = 2
    sample_num = 1000
    sample_x, normal, rprime = Ds.petaldomain(sample_num)

sample_u_r,sample_u_i,sample_xk = Bdc.exact_sol_operator(sample_x,k,example)


In [154]:
# Generate integral matrix
"""
r0 = torch.zeros(sample_num,sample_num)
n1 = torch.zeros(sample_num,sample_num)
c = torch.zeros(sample_num,sample_num)
KaparR2 = torch.ones(sample_num)
KaparR2[1],KaparR2[2] =KaparR2[1] + 1.825748064736159, KaparR2[2] - 1.325748064736159
j_index = np.array([i for i in range(sample_num)])
for i in range(sample_num):
    d = sample_x-sample_x[i,:]
    r0[i,:] = d.norm(dim=1)
    n1[i,:] = (normal*d).sum(axis=1)
    c[i,:] = KaparR2[abs((j_index-i+round(sample_num/2))%sample_num-round(sample_num/2))]*((rprime).norm(dim=1))/sample_num*2*np.pi

G2_r = torch.zeros(k_num,sample_num,sample_num)
G2_i = torch.zeros(k_num,sample_num,sample_num)
for l in range(k_num):
    G2_r[l,:,:] = scp.hankel1(1,k[l]*r0).imag/4*n1/r0*k[l]*(-c)
    G2_i[l,:,:] = -scp.hankel1(1,k[l]*r0).real/4*n1/r0*k[l]*(-c)
    G2_r[l,j_index,j_index],G2_i[l,j_index,j_index] = -1/2,0
"""
G_r = torch.zeros(k_num,sample_num,sample_num)
G_i = torch.zeros(k_num,sample_num,sample_num)
for l in range(k_num):
    G_r[l],G_i[l] = IM.G_bd(domain_type,problem_type+str(k[l].numpy()),layer,sample_x,rprime,normal,in_or_out)

In [68]:
net1 = Nw.ResNet2(100,3,2).to(device)
loss_func = lf.Helmholtz_loss()

In [166]:
# Training process
optimizer = torch.optim.Adam(net1.parameters(net1),lr=0.0005)
Epoch, each_epoch = 100, 10
for i in range(2):
    k = (torch.cat((torch.rand(20)*1.5+2,torch.rand(40)*1.5+4.5),0)).to(device)
    sample_u_s,sample_u_i,sample_xk1 = Bdc.exact_sol_operator(sample_x,k,example)
    for l in range(k_num):
        G_r[l],G_i[l] = IM.G_bd(domain_type,'Helmholtz'+str(k[l].numpy()),layer,sample_x,rprime,normal,in_or_out)
    G_r,G_i = G_r.to(device), G_i.to(device)
    sample_u_r,sample_u_i,sample_xk1 = sample_u_r.to(device),sample_u_i.to(device),sample_xk1.to(device)
    
    net1,loss = Tn.Helmholtz_training_operator(net1,k,sample_xk,sample_u_r,sample_u_i,G_r,G_i,loss_func,optimizer,Epoch,each_epoch)


loss, epoch, computation time: 5.4377 0 0.7031
loss, epoch, computation time: 0.9716 10 6.6103
loss, epoch, computation time: 0.7887 20 7.0003
loss, epoch, computation time: 0.6571 30 6.5206
loss, epoch, computation time: 0.6520 40 6.6512
loss, epoch, computation time: 0.6293 50 6.9075
loss, epoch, computation time: 0.6251 60 6.9195
loss, epoch, computation time: 0.6189 70 6.7769
loss, epoch, computation time: 0.6151 80 6.8118
loss, epoch, computation time: 0.6122 90 6.9833
loss, epoch, computation time: 0.6095 100 6.7260
loss, epoch, computation time: 1.0401 0 0.6722
loss, epoch, computation time: 0.6177 10 7.1549
loss, epoch, computation time: 0.5203 20 7.5438
loss, epoch, computation time: 0.5080 30 7.4291
loss, epoch, computation time: 0.5057 40 7.1579
loss, epoch, computation time: 0.5001 50 7.6037
loss, epoch, computation time: 0.4966 60 7.4581
loss, epoch, computation time: 0.4946 70 7.6675
loss, epoch, computation time: 0.4927 80 8.1203
loss, epoch, computation time: 0.4911 90 

In [172]:
# Test the accuracy

# sample          The number of the sample points in the domain 
# x_in            The coordinates of the sample points
# u_in_r, u_in_i  The exact solution of the sample points
# G_in_r, G_in_i  The integral matrix for the points in the domain 

sample = 10
theta_in = torch.rand(sample,1)*2*np.pi
r_in = torch.rand(sample,1)*(2-(9/20-1/9*torch.cos(5*theta_in)))+(9/20-1/9*torch.cos(5*theta_in))*1.01
x_in = torch.cat((torch.cos(theta_in), torch.sin(theta_in)),1)*r_in 

rprime = rprime.to(device)
normal = normal.to(device)

r0_in = (torch.zeros(sample,sample_num,dtype=torch.float32)).to(device)
c_in = (torch.zeros(sample,sample_num,dtype=torch.float32)).to(device)
for i in range(sample):
    for j in range(sample_num):
        d = sample_x[j,:]-x_in[i,:]
        r0_in[i,j] = d.norm()
        c_in[i,j] = (rprime[j,:].norm())/sample_num*2*np.pi*((d*normal[j,:]).sum())/r0_in[i,j]

# k_test  The wavenumbers for test error
# l2_all  The relative L2 error with different k
k_test = 70
kx = torch.zeros(k_test+1)
l2_all = torch.zeros(k_test+1)
for i in range(k_test+1):
    k = i/k_test*7+1

    u_in_r = (scp.hankel1(0,(k*torch.sqrt((x_in[:,0])**2+x_in[:,1]**2)).cpu()).real).to(device) 
    u_in_i = (scp.hankel1(0,(k*torch.sqrt((x_in[:,0])**2+x_in[:,1]**2)).cpu()).imag).to(device) 

    G2_in_r = (scp.hankel1(1,(k*r0_in).cpu()).imag).to(device)/4*c_in*k
    G2_in_i = -(scp.hankel1(1,(k*r0_in).cpu()).real).to(device)/4*c_in*k
    x_sample_in = (torch.zeros(sample_num,3,dtype=torch.float32)).to(device)
    x_sample_in[:,0:2] = sample_x
    x_sample_in[:,2] = k
    sample_h_r,sample_h_i = net1(x_sample_in)[:,0],net1(x_sample_in)[:,1]

    u_green_r = -(G2_in_r@sample_h_r-G2_in_i@sample_h_i)
    u_green_i = -(G2_in_r@sample_h_i+G2_in_i@sample_h_r)
    kx[i] = k
    l2_all[i] = (((u_green_r-u_in_r)**2+(u_green_i-u_in_i)**2).sum()/((u_in_r)**2+(u_in_i)**2).sum()).sqrt()
